In [1]:
import pandas as pd
from pandas.io import gbq

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#### Get Spotify playlists of interest

In [ ]:
get_throwback_in_uk_browse = """
SELECT
  category_name,
  playlist_id,
  playlist_name
FROM
  `umg-alpha.spotify.spotify_playlist_browse`
WHERE
  owner_id = "spotify"
  AND territory = "GB"
GROUP BY
  1,
  2,
  3
"""

#### Get the demographics that are listening to these Spotify playlists

Rank each demographic audience by the number of streams they do from each playlist

In [ ]:
get_spotify_demographics = """
WITH
  playlists AS (
  SELECT
    category_name,
    playlist_id,
    playlist_name
  FROM
    `umg-comm-tech-dev.playlist_pitching_analytics.top_browse_uk_playlists`
  GROUP BY
    1,
    2,
    3)
SELECT
  category_name,
  playlist_id,
  playlist_name,
  user_country_name,
  user_gender,
  user_age,
  s.user_age_group,
  COUNT(*) AS streams,
  COUNT(DISTINCT user_id) AS listeners
FROM
  playlists p
LEFT JOIN
  `umg-edw.spotify.streams` s
ON
  p.playlist_id=REGEXP_EXTRACT(s.source_uri,r'playlist:(.*)')
WHERE
  s.stream_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 21 DAY)
  AND s.stream_date < CURRENT_DATE()
  AND s.user_country_code = "GB"
  AND s.user_gender != ''
  AND s.user_age IS NOT NULL
GROUP BY
  category_name,
  playlist_id,
  playlist_name,
  user_country_name,
  user_gender,
  user_age,
  s.user_age_group
"""

In [ ]:
spotify_demographic_groups = """
SELECT
  playlist_id,
  playlist_name,
  user_gender,
  user_age_group,
  SUM(streams) AS streams,
  SUM(listeners) AS listeners,
  RANK() OVER(PARTITION BY playlist_name ORDER BY SUM(streams) DESC) AS rank_streams
FROM
  `umg-comm-tech-dev.playlist_pitching_analytics.demographics_spotify`
WHERE
  user_gender != ''
  AND user_age IS NOT NULL
GROUP BY
  playlist_id,
  playlist_name,
  user_gender,
  user_age_group
"""

In [ ]:
sp_demographics = gbq.read_gbq(spotify_demographic_groups, project_id='umg-comm-tech-dev', dialect='standard')

In [ ]:
sp_demographics.head()

In [ ]:
sp_demographics=sp_demographics[sp_demographics.user_age_group!="Unknown"]

#### Checking the audiences for these playlists and their relative distribution

In [ ]:
by_gender_age = sp_demographics.groupby(by=['user_gender','user_age_group'])

In [ ]:
df_dem_sp = pd.DataFrame(by_gender_age.streams.sum())
df_dem_sp['listeners'] = by_gender_age.listeners.sum()
df_dem_sp['playlist_listens']=by_gender_age.playlist_id.nunique()

In [ ]:
df_dem_sp = df_dem_sp.reset_index()
df_dem_sp.head()

In [ ]:
fig = plt.figure(figsize=(16,8))

cmap = sns.cubehelix_palette(dark=.3, light=.8, as_cmap=True)
sns.scatterplot(data=df_dem_sp, x='user_age_group', y='streams', size='listeners', sizes=(50,250),
                style='user_gender', hue='listeners', legend="full",
                palette="Set2")

plt.show()

Demographics is nostly as expected. Most numerous audiences are both men and women between 25-34, then 18-24 (notably girls more than boys), then 35-44s, 45-54th, etc. Interestingly 45-54 year olds have the same amount opf listeners as 55-64 year olds, but the former generate significantly more streams.

#### Get the ranking of ALL artists on Apple for each of our demographic groups...

In [ ]:
apple_artist_ranks_global = """
SELECT
  apple_artist_id,
  artist_name,
  user_gender,
  user_age_group,
  SUM(stream_count) AS streams,
  SUM(listener_count) AS listeners,
  RANK() OVER(PARTITION BY user_gender,user_age_group ORDER BY SUM(stream_count) DESC) AS artist_rank
FROM
  `umg-edw.apple_music.daily_artist_demographics`
WHERE stream_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY)
  AND stream_date < CURRENT_DATE()
  AND user_country_code ="GB"
  GROUP BY
  apple_artist_id,
  artist_name,
  user_gender,
  user_age_group
"""

#### ...and then select our top 80 catalogue artists, their rankings for each demographic and their tracks

In [ ]:
apple_artist_content_demographic = """
SELECT
  artist_id,
  isrc,
  title,
  d.artist_name,
  d.user_gender,
  d.user_age_group,
  r.artist_rank,
  SUM(listener_count) AS listener_count,
  SUM(stream_count) AS stream_count
FROM
  `umg-edw.apple_music.daily_content_demographics` d
LEFT JOIN
  `umg-comm-tech-dev.playlist_pitching_analytics.artist_global_ranking` r
ON
  d.artist_id = r.apple_artist_id
  AND d.user_gender = r.user_gender
  AND d.user_age_group = r.user_age_group
WHERE
  stream_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 180 DAY)
  AND stream_date < CURRENT_DATE()
  AND artist_id IN (
  SELECT
    artist_id_apple
  FROM
    `umg-comm-tech-dev.hundred_artist.joined_artists_track_data`
  GROUP BY
    artist_id_apple)
  AND user_country_code = "GB"
GROUP BY
  artist_id,
  isrc,
  title,
  artist_name,
  user_gender,
  user_age_group,
  r.artist_rank 
"""

In [ ]:
global_ranking = """
SELECT
  *,
IF
  (apple_artist_id IN (
    SELECT
      artist_id_apple
    FROM
      `umg-comm-tech-dev.hundred_artist.joined_artists_track_data`
    GROUP BY
      artist_id_apple),
    1,
    0) AS in_analysis
FROM
  `umg-comm-tech-dev.playlist_pitching_analytics.artist_global_ranking`
"""

apple_demographic = gbq.read_gbq(global_ranking, project_id='umg-comm-tech-dev', dialect='standard')
apple_demographic.head()

In [ ]:
df_apple = apple_demographic[(apple_demographic.user_gender!='Data Unknown')&(apple_demographic.user_age_group!='Data Unknown')]

#### Let's check how popular are our top80 catalogue artists in each of the demographic

In [ ]:
fig = plt.figure(figsize=(16,8))

sns.boxplot(data = df_apple[df_apple.in_analysis==1], x='artist_rank', y='user_age_group', hue='user_gender',
           palette="Set2")

plt.show()

It's fair to say that our artists are quite popular equally in all the demographics. 75% of those artists are ranked under 250 in all demographics. The least liked artist by 45-54s that has the ranking of lower than 5000s in their group is Eason Chan :)

![](8ed.jpg)

In [ ]:
df_apple[(df_apple.in_analysis==1)&(df_apple.artist_rank>5000)]

Below is the distribution of ALL artists on Apple ranking within each demographic. Safe to say our top 80 are doing better than average

In [ ]:
fig = plt.figure(figsize=(16,8))

sns.boxplot(data = df_apple, x='artist_rank', y='user_age_group', hue='user_gender',
           palette="Set3")

plt.show()

#### Getting all our artists and their songs where their respective rank in each demographic group is no lower than 250

In [ ]:
our_ranking = """
SELECT
  *
FROM
  `umg-comm-tech-dev.playlist_pitching_analytics.apple_content_demographic`
WHERE
  artist_rank <= 250
  AND user_gender != 'Data Unknown'
  AND user_age_group != 'Data Unknown'
"""

apple_demographic_tracks = gbq.read_gbq(our_ranking, project_id='umg-comm-tech-dev', dialect='standard')
apple_demographic_tracks.head()

In [ ]:
sp_demographics.head()

In [ ]:
sp_demographics.user_age_group.unique()

In [ ]:
sp_demographics.user_gender.unique()

In [ ]:
apple_demographic_tracks.user_age_group.unique()

In [ ]:
apple_demographic_tracks['user_gender'] = [str(x).lower() for x in apple_demographic_tracks.user_gender]

In [ ]:
apple_demographic_tracks.user_gender.unique()

#### Getting together Spotify playlist demographics with apple demographics and our top80 catalogue artists' rep

In [ ]:
df_all = pd.merge(sp_demographics,apple_demographic_tracks, on = ['user_gender', 'user_age_group'])
df_all.head()

In [ ]:
df_all.shape

In [ ]:
df_all.to_csv('df_all.csv', sep='\t')

In [4]:
df_all = pd.read_csv('df_all.csv', sep='\t', index_col='Unnamed: 0')

/usr/local/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
df_all.head()

,playlist_id,playlist_name,user_gender,user_age_group,streams,listeners,rank_streams,artist_id,isrc,title,artist_name,artist_rank,listener_count,stream_count
0,37i9dQZF1DX1OY2Lp0bIPp,#MondayMotivation,female,25-34,595940,36506,1,159260351,GBUM70900581,Should've Said No(International Mix),Taylor Swift,3,7117,8204
1,37i9dQZF1DX1OY2Lp0bIPp,#MondayMotivation,female,25-34,595940,36506,1,159260351,GBUM70900582,Love Story(Stripped),Taylor Swift,3,5,5
2,37i9dQZF1DX1OY2Lp0bIPp,#MondayMotivation,female,25-34,595940,36506,1,159260351,GBUM70900586,Teardrops On My Guitar,Taylor Swift,3,5,6
3,37i9dQZF1DX1OY2Lp0bIPp,#MondayMotivation,female,25-34,595940,36506,1,159260351,GBUM70900586,Teardrops On My Guitar(International Mix),Taylor Swift,3,7248,8395
4,37i9dQZF1DX1OY2Lp0bIPp,#MondayMotivation,female,25-34,595940,36506,1,159260351,GBUM71026563,Mine,Taylor Swift,3,7826,9267


In [ ]:
df_all.to_gbq(destination_table='playlist_pitching_analytics.apple_suggestions', project_id='umg-comm-tech-dev',
             if_exists='replace')

0it [00:00, ?it/s]

#### Now checking which of these tracks and artists are already in those respective Spotify playlists

In [ ]:
check_track_listings = """
SELECT
  playlist_id,
  track_artist,
  track_title,
  isrc
FROM
  `umg-edw.spotify.playlist_track_history`
WHERE
  playlist_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 180 DAY)
  AND playlist_date < CURRENT_DATE()
  AND playlist_id IN (
  SELECT
    playlist_id
  FROM
    `umg-comm-tech-dev.playlist_pitching_analytics.top_browse_uk_playlists`
  GROUP BY
    1)
GROUP BY
  playlist_id,
  track_artist,
  track_title,
  isrc
"""

In [ ]:
track_listings = gbq.read_gbq(check_track_listings, project_id='umg-comm-tech-dev', dialect='standard')
track_listings['isrc_in_playlist'] = 1
track_listings.head()

In [ ]:
df_all_tracks = pd.merge(df_all, track_listings, on=['playlist_id','isrc'], how='left')
df_all_tracks['isrc_in_playlist'] = df_all_tracks.isrc_in_playlist.fillna(int(0))
df_all_tracks.head()

In [ ]:
by_pl_art = df_all_tracks.groupby(by=['playlist_id','artist_name']).isrc_in_playlist.max()
df_pl_art = pd.DataFrame(by_pl_art).reset_index()
df_pl_art.columns = ['playlist_id','artist_name','artist_in_playlist']
df_pl_art.head()

In [ ]:
df_pl_art[df_pl_art.playlist_id=='0ApdHY8K71F9WrIWbgiI2G']

In [ ]:
df_final = pd.merge(df_all_tracks, df_pl_art, on=['playlist_id','artist_name'], how='left')
df_final.head()

In [ ]:
df_final[(df_final.playlist_id=='0ApdHY8K71F9WrIWbgiI2G')&(df_final.artist_name=="The Beatles")]

In [ ]:
df_final.to_gbq(destination_table='playlist_pitching_analytics.final_apple_suggestions', project_id='umg-comm-tech-dev',
             if_exists='replace')

### And now over to the dashboard!
![](https://media.giphy.com/media/236WrWu95GphLreE2i/giphy.gif)